In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [5]:
data = pd.read_csv("D:/data.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:1024]
Y = Y[:, 1024]

X_train = X[0:70000, :]
X_train = X_train / 255.
X_test = X[70000:72001, :]
X_test = X_test / 255.

Y = Y.reshape(Y.shape[0], 1)   #Reshape
Y_train = Y[0:70000, :]
Y_train = Y_train.T
Y_test = Y[70000:72001, :]
Y_test = Y_test.T

In [6]:
print("Number of training examples : " + str(X_train.shape[0]))
print("Number of test examples : " + str(X_test.shape[0]))
print("X_train shape : " + str(X_train.shape))
print("Y_train shape : " + str(Y_train.shape))
print("X_test shape : " + str(X_test.shape))
print("Y_test shape : " + str(Y_test.shape))

Number of training examples : 70000
Number of test examples : 2000
X_train shape : (70000, 1024)
Y_train shape : (1, 70000)
X_test shape : (2000, 1024)
Y_test shape : (1, 2000)


In [7]:
image_x = 32
image_y = 32

train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], image_x, image_y, 1)
X_test = X_test.reshape(X_test.shape[0], image_x, image_y, 1)

In [9]:
print("X_train shape : " + str(X_train.shape))
print("Y_train shape : " + str(train_y.shape))

X_train shape : (70000, 32, 32, 1)
Y_train shape : (70000, 37)


In [12]:
#Building a model
def keras_model(image_x, image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), input_shape = (image_x, image_y, 1), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = "same"))
    model.add(Conv2D(64, (5, 5), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (5, 5), strides = (5, 5), padding = "same"))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation = "softmax"))
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    filepath = "devanagari.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor = "val_acc", verbose = 1, save_best_only = True, mode = "max")
    callbacks_list = [checkpoint1]
    
    return model, callbacks_list

In [13]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data = (X_test, test_y), epochs = 8, batch_size = 64, callbacks = callbacks_list)
scores = model.evaluate(X_test, test_y, verbose = 0)
print("CNN Error : %.2f%%" % (100- scores[1] * 100))
print_summary(model)
model.save("devanagari.h5")

Instructions for updating:
Use tf.cast instead.
Train on 70000 samples, validate on 2000 samples
Epoch 1/8
70000/70000 [==============================] - 178s 3ms/step - loss: 0.8011 - acc: 0.7717 - val_loss: 0.3351 - val_acc: 0.9035

Epoch 00001: val_acc improved from -inf to 0.90350, saving model to devanagari.h5
Epoch 2/8
70000/70000 [==============================] - 167s 2ms/step - loss: 0.2406 - acc: 0.9264 - val_loss: 0.2024 - val_acc: 0.9410

Epoch 00002: val_acc improved from 0.90350 to 0.94100, saving model to devanagari.h5
Epoch 3/8
70000/70000 [==============================] - 165s 2ms/step - loss: 0.1558 - acc: 0.9521 - val_loss: 0.2152 - val_acc: 0.9335

Epoch 00003: val_acc did not improve from 0.94100
Epoch 4/8
70000/70000 [==============================] - 164s 2ms/step - loss: 0.1158 - acc: 0.9637 - val_loss: 0.1206 - val_acc: 0.9665

Epoch 00004: val_acc improved from 0.94100 to 0.96650, saving model to devanagari.h5
Epoch 5/8
70000/70000 [==========================